# 앙상블(ensemble)

## 배깅(bagging)
```from sklearn.ensemble import BaggingClassifier(base_estimator, n_estimators, max_samples, max_features, oob_score)```
* best_estimator: 배깅에서 수행할 분류기. None이면 DecisionTreeClassifier를 수행
* n_estimators: 앙상블의 분류기 모델 수를 설정
* max_samples: 각 기본 추정량을 훈련하기 위해 추출할 샘플의 수 (비율, default=1)
* max_features: 하나의 예측기가 가져갈 수 있는 최대 변수의 수
* oob_score: 일반화 오류 추정을 위한 oob 샘플 사용 여부


| 주어진 자료에서 여러 개의 붓스트랩 자료를 생성하고 각각에 예측 모형을 만든 후 결합하고 최종 예측모형을 만드는 방식
* 분산을 줄이고 정확도를 개선하여 과적합을 방지한다.
* 각 붓스트랩에 대해 붓스트래핑 및 모델링 과정에 병렬적으로 수행된다.

[붓스트랩 - 모델링 - 보팅] 의 순서로 진행

**붓스트랩(Bootstrap)**
* 랜덤 샘플링의 일종으로 가설 검증을 하거나 통계 계산을 하기 전에 단순임의복원추출법(중복허용)을 적용하여 여러 개의 동일한 크기의 표본 자료를 획득하는 방법

    주어진 데이터를 모집단을 대표하는 독립 표본으로 가정하고, 무작위 재주출을 하여 복수의 자료를 획득하고 각각의 통게량을 계산

**보팅(Voting)**
* 최적 트리 구축에서 가장 어려운 가지치기를 진행하지 않고 약한 트리를 최대로 성장시킨 후 보팅을 진행


**OOB_score**

* 배깅에서 붓스트래핑 진행 시 평균적으로 훈련데이터의 63%정도만 샘플린
* 이때 선택되지 않은 나머지 37%가 **Out Of Bag** 샘플

    모델의 성능평가 수행 가능 -> 검증세트가 따로 필요하지 않음

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import *


### 배깅 예제 - 유방암 진단 데이터 (분류)
❗️회귀 모델에서 변수선택법 실습해보기❗️

In [2]:
breast = pd.read_csv('https://raw.githubusercontent.com/ADPclass/ADP_book_ver01/main/data/breast-cancer.csv')
breast.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
# 결측치
breast.isna().sum().loc[breast.isna().sum()!=0]

Series([], dtype: int64)

In [6]:
# 이상치
breast.duplicated().sum()

0

In [3]:
breast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [13]:
# 타겟 분포 확인 (M: 악성, B: 양성)
breast.diagnosis.value_counts(normalize=True)

diagnosis
0    0.627417
1    0.372583
Name: proportion, dtype: float64

In [10]:
# 인코딩
breast.diagnosis.value_counts(normalize=True)
breast.diagnosis = np.where(breast.diagnosis == 'M', 1, 0)


In [18]:
corr = breast.drop('id', axis=1)

pos_corr = corr.corr(method='spearman').sort_values('diagnosis', ascending=False)[:10]
neg_corr = corr.corr(method='spearman').sort_values('diagnosis')[:5]

In [19]:
pos_corr

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
diagnosis,1.000000,0.732785,0.461971,0.748496,0.734122,0.371892,0.609288,0.733308,0.777877,0.332567,...,0.787933,0.476720,0.796319,0.786902,0.425513,0.606810,0.705734,0.781674,0.396843,0.311477
perimeter_worst,0.796319,0.971555,0.375273,0.978980,0.971822,0.226345,0.592254,0.722424,0.813960,0.190526,...,0.993548,0.381022,1.000000,0.992433,0.241172,0.613070,0.700572,0.812983,0.281383,0.179003
radius_worst,0.787933,0.978604,0.366547,0.981244,0.979258,0.203453,0.542626,0.682316,0.787411,0.164552,...,1.000000,0.371230,0.993548,0.998891,0.218616,0.558316,0.655942,0.780632,0.257165,0.127449
area_worst,0.786902,0.978863,0.368335,0.980864,0.980264,0.191735,0.531590,0.676628,0.780395,0.154462,...,0.998891,0.372376,0.992433,1.000000,0.210063,0.550007,0.651120,0.773945,0.248358,0.118734
concave points_worst,0.781674,0.727265,0.319235,0.757526,0.723390,0.498868,0.825473,0.904938,0.937075,0.397477,...,0.780632,0.365309,0.812983,0.773945,0.543982,0.844454,0.902301,1.000000,0.460711,0.516664
concave points_mean,0.777877,0.759702,0.306891,0.788629,0.755165,0.565172,0.848295,0.927352,1.000000,0.423767,...,0.787411,0.300562,0.813960,0.780395,0.490035,0.758309,0.827281,0.937075,0.355477,0.421110
perimeter_mean,0.748496,0.997802,0.348142,1.000000,0.997068,0.182923,0.543925,0.681958,0.788629,0.150049,...,0.981244,0.323109,0.978980,0.980864,0.156611,0.534565,0.632106,0.757526,0.199007,0.088961
area_mean,0.734122,0.999602,0.344145,0.997068,1.000000,0.138053,0.488988,0.642557,0.755165,0.113928,...,0.979258,0.318178,0.971822,0.980264,0.119712,0.485813,0.593736,0.723390,0.170860,0.038758
concavity_mean,0.733308,0.645728,0.342646,0.681958,0.642557,0.518511,0.896518,1.000000,0.927352,0.446793,...,0.682316,0.335866,0.722424,0.676628,0.488775,0.849985,0.938543,0.904938,0.383667,0.541838
radius_mean,0.732785,1.000000,0.340956,0.997802,0.999602,0.148510,0.497578,0.645728,0.759702,0.120242,...,0.978604,0.314911,0.971555,0.978863,0.125789,0.491357,0.596043,0.727265,0.174698,0.044564


In [23]:
neg_corr.iloc[:, 0]

symmetry_se              -0.092303
smoothness_se            -0.052193
fractal_dimension_mean   -0.025903
texture_se                0.019419
fractal_dimension_se      0.201492
Name: diagnosis, dtype: float64

In [24]:
def metrics(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    rc = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    print(f'''
          혼동행렬
          {cm}
          recall: {rc}
          f1_score: {f1}''')
    

In [26]:
# 상관관계가 높은 변수 10개 선택
x = breast[pos_corr.index[1:]]
y = breast.diagnosis
# 데이터 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.3, random_state=1, stratify=y)
# 학습
clf = BaggingClassifier(n_estimators=30, oob_score=True, max_samples=.9)
clf.fit(x_train, y_train)
# 학습 성능 -> 혼동행렬 recall f1
y_pred = clf.predict(x_train)
metrics(y_train, y_pred)




          혼동행렬
          [[250   0]
 [  1 147]]
          recall: 0.9932432432432432
          f1_score: 0.9966101694915254


In [ ]:
# oob_score 확인
# 일반화 성능 확인 